In [1]:
## importing necessary libraries
import pandas as pd
from datasets import load_dataset

Downloading banking queries data from Huggingface 🤗

In [2]:
dataset  = load_dataset('banking77')

Using custom data configuration default
Reusing dataset banking77 (C:\Users\barka\.cache\huggingface\datasets\banking77\default\1.1.0\aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


  0%|          | 0/2 [00:00<?, ?it/s]

## Making dataframe and storing as csv file

In [25]:
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame( dataset['test'])

In [26]:
train_df.to_csv('./input/train.csv',index=False)
test_df.to_csv('./input/test.csv',index=False)

## Initializing W&B run 

In [7]:
import wandb
PROJECT_NAME = "banking_77"
JOB_TYPE = "preprocess"
ENTITY = "basha"

run = wandb.init(project=PROJECT_NAME,job_type=JOB_TYPE,entity=ENTITY)

## Creating Artifact(folder)
In the below code snippet we are creating a folder in Artifact and uploading our train and test csv file in W&B. This will help us to use same dataset through our project.

In [18]:
artifact = wandb.Artifact("RAW_DATA",type="dataset")

In [19]:
artifact.add_dir('./input')

wandb: Adding directory to artifact (.\input)... Done. 0.0s


In [27]:
run.log_artifact('./input',"RAW_DATA",type="dataset")

wandb: Adding directory to artifact (.\input)... Done. 0.0s


In [81]:
run.finish()

## Creating Training and validation dataset

In [82]:
JOB_TYPE = "data_split"
run = wandb.init(project=PROJECT_NAME,job_type=JOB_TYPE,entity=ENTITY)

In [83]:
artifact = run.use_artifact('basha/banking_77/RAW_DATA:latest', type='dataset')
artifact_dir = artifact.download()

In [84]:
train_df = pd.read_csv(artifact_dir+"\\train.csv")
test_df = pd.read_csv(artifact_dir+"\\test.csv")

We are spliting our training set into 2 sets, train and validation set. We are using sklearn train_test_split method and applying stratify on labels

In [85]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(train_df['text'],train_df['label'],stratify=train_df['label'],test_size=0.1,random_state=42)

## Creating pandas Dataframe and saving as CSV file

In [86]:
train_split = pd.DataFrame({'text': X_train.values,'label' : y_train.values})
valid_split = pd.DataFrame({'text': X_valid.values,'label' : y_valid.values})

In [87]:
train_split.to_csv('./split/train_split.csv',index=False)
valid_split.to_csv('./split/valid_split.csv',index=False)
test_df.to_csv('./split/test_split.csv',index=False)

## Uploading train, test and validation set to W&B

In [88]:
split_artifact = wandb.Artifact('preprocess',type='split_data')
# split_artifact.add_dir('./split')

In [89]:
split_artifact.add_file('./split/train_split.csv')
split_artifact.add_file('./split/test_split.csv')
split_artifact.add_file('./split/valid_split.csv')

<ManifestEntry digest: idV++gGUhgcg6Ed+QJPlXw==>

In [90]:
split_artifact.add_dir(artifact_dir)

wandb: Adding directory to artifact (.\artifacts\RAW_DATA-v1)... Done. 0.0s


In [91]:
run.log_artifact(split_artifact)

In [92]:
run.finish()